In [19]:
import redis
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
import time
import datetime

# Setup
functions for connecting and setting up the databases 

In [20]:
current_env = 'production' # 'docker' for development on docker, 'production' for real server
current_user = 'stud10'
ENV = {"docker": '172.17.0.1',"production":'bdl1.eng.tau.ac.il'}
companiesSet = current_user+":company:names"
ojOSet = current_user+":oj"
cand_app_set = current_user+":candidate_application"

### Connections

In [21]:
def connect_to_redis(port=6379):
    r = redis.StrictRedis(host=ENV[current_env], port=port, socket_connect_timeout=10)
    r.ping() # send ping to verify that a connection established to redis
    print('connected to redis on "{}", port {}'.format(ENV[current_env], port)) 
    return r

def connect_to_mongo(port=27017):
    if current_env == 'docker':
        client = MongoClient(host=ENV[current_env],port=port, connectTimeoutMS=10000)
        # client.admin.command('ping') # try to ping mongo server
        print('connected to MongoDB on "{}", port {}'.format(ENV[current_env], port))
    
    elif current_env == 'production':
        client = MongoClient()
    return client


### Initializiation 

In [22]:
def setup_mongo(client):
    db = client[current_user]
    companies = db.companies
    print("companies collection created in {} db".format(current_user))
    return db, companies

def reset_redis(r):
    print("deleting companies set ({})".format(companiesSet))
    r.delete(companiesSet)
    print("deleting open jobs ({})".format(ojOSet))
    r.delete(ojOSet)
    print("deleting candidate  ({})".format(ojOSet))
    r.delete(ojOSet)
    print("redis is clean") #TODO: check if specific delete is required

def reset_mongo(client):
    client[current_user].companies.drop()
    print("mongo is clean")

def restart():
    # connect to dbs
    r = connect_to_redis()
    client = connect_to_mongo()
    # clean dbs
    reset_redis(r)
    reset_mongo(client)
    # setup the databases
    db, companies = setup_mongo(client)
    print("restart completed")
    return r, client, db, companies

# Operation 1 - Add a new company

In [23]:
def is_company_exists(r, companies, company_name):
    # check if company name in comapny set on redis
    if r.sismember(companiesSet, company_name):
        return True
    
    # if not, verify against mongo
    elif companies.find_one({"company_name": company_name}) is not None:
        return True
    
    else:
        return False
    
def add_company(company_dict,r=None, companies=None):
    if r is None:
        r = connect_to_redis()
    if companies is None:
        companies = connect_to_mongo()[current_user].companies
        
    # getting dict for the company and insert it to db
    if 'company_name' not in company_dict:
        # raise ValueError("company dict must have company_name field") #no errors allowed
        print("company dict must have company_name field, no changes were commited to db")
        return None

    else:
        company_name = company_dict['company_name']
    
    # verify that company name is unique
    if(is_company_exists(r, companies, company_name)):
        # raise ValueError("company name already taken") #no errors allowed
        print("company name already taken, no changes were commited to db")
        return None
        
    # and then insert to mongo and redis
    company_dict['jobs_list'] = [] # initiate jobs list
    companies.insert_one(company_dict)
    r.sadd(companiesSet, company_dict['company_name'])
    print("%s Added successfully!"%(company_name))

# Operation 2 - Add a new job position

In [24]:
def generate_job_id(companies, company_name):
    match = {'$match' : {'company_name':company_name}}
    project = {'$project': { 'max_id': { '$size':'$jobs_list' }}}
    res = companies.aggregate([match,project])
    return list(res)[0]['max_id'] + 1

def add_job(job_dict, company_name, r=None, companies=None):
    # ASSUMPTION: there is no option to delete jobs (so counting jobs can be used to generate job id)
    if r is None:
        r = connect_to_redis()
    if companies is None:
        companies = connect_to_mongo()[current_user].companies
    if not (is_company_exists(r, companies, company_name)):
        raise ValueError("company doesn't exist")
    
    # generate job id and insert to the company object
    job_dict['job_id'] = generate_job_id(companies, company_name)
    job_dict['application_list'] = [] # initiate application list
    companies.update_one({'company_name': company_name}, {'$push': {'jobs_list': job_dict}}, upsert = True)
    r.zincrby(ojOSet,"%s:%s"%(job_dict['location'],job_dict['job_title']),amount=1)
    print("job with id %s was added to %s jobs successfully!"%(job_dict['job_id'], company_name))

# Operation 3 - Add a new application

In [25]:
def is_job_open(companies, company_name, job_id):
    res = companies.find_one(
        { 
            "company_name": company_name,
        },
        { "jobs_list": { "$elemMatch": { "job_id": int(job_id), "status": "open" }}}
    )
    return('jobs_list' in res)

def is_already_submitted(companies, company_name, job_id, email):
    res = companies.find_one(
        { 
            "company_name": company_name,
        },
        { "jobs_list": { "$elemMatch": { "job_id": int(job_id), "application_list": {"$elemMatch": {'email':email}} }}}
    )
    return('jobs_list' in res) # if mail doesn't exists an empty object will returned

def new_application(candidate, application_time, job_id, company_name, r=None, companies=None):
    if r is None:
        r = connect_to_redis()
    if companies is None:
        companies = connect_to_mongo()[current_user].companies

    if (not is_job_open(companies, company_name, job_id)):
        print("you are trying to apply to a closed job")
        return -1

    if (is_already_submitted(companies, company_name, job_id, candidate['email'])):
        print("you have already sent application for this job")
        return -2

    # update in mongo
    d = datetime.datetime.strptime(application_time, "%d-%m-%Y %H:%M:%S")
    candidate['application_date'] = d
    companies.update_one({"company_name": company_name,"jobs_list": {"$elemMatch":{"job_id":int(job_id)}}},{'$push':{'jobs_list.$.application_list':candidate}})

    #update in redis
    unix_d = time.mktime(d.timetuple())
    #check if company already exists in redis
    key_string = "candidate_applications:{}".format(candidate['email'])
    comps = r.zrange(key_string,0,-1,withscores=True)
    comp = [x for x in comps if x[0]==company_name]
    # if comany already exists - change vlaue to most recent date
    if len(comp)>0:
        old_timestamp = comp[0][1]
        if unix_d>old_timestamp:
            r.zadd(key_string,unix_d,company_name)
    else:
        r.zadd(key_string,unix_d,company_name)
        
    print("{} submited application for job number {} at {}".format(candidate['candidate_name'],job_id, company_name))

# Operation 4 - Update job status

In [26]:
def update_job_status(company_name, job_id, new_status, r=None, companies=None):
    #TODO: update open jobs on redis
    if r is None:
        r = connect_to_redis()
    if companies is None:
        companies = connect_to_mongo()[current_user].companies
    
    companies.update_one({"company_name": company_name,"jobs_list": {"$elemMatch":{"job_id":int(job_id)}}},{'$set':{'jobs_list.$.status':new_status}})
    print("job number: {} at {} is now: {}".format(job_id, company_name, new_status))

# Operation 5 - show latest companies

In [27]:
def show_latest_10_companies(candidate_email):
    key_string = "candidate_applications:{}".format(candidate_email)
    return r.zrevrange(key_string, 0, 9, withscores=False)

# Operation 6 - show number of open jobs

In [44]:
def show_number_of_jobs(location,title):
    key_string = "%s:%s"%(location,title)
    results = r.zrevrange(ojOSet, 0, -1, withscores=True)
    for res in results:
        if res[0] == key_string:
            return res[1]
    return 0

# Run all operations

In [29]:
r = connect_to_redis()
client = connect_to_mongo()

# clean the databases
reset_redis(r)
reset_mongo(client)

# setup the databases
db, companies = setup_mongo(client)

connected to redis on "bdl1.eng.tau.ac.il", port 6379
deleting companies set (stud10:company:names)
deleting open jobs (stud10:oj)
deleting candidate  (stud10:oj)
redis is clean
mongo is clean
companies collection created in stud10 db


In [30]:
# Operation 1 - Add a new company
add_company({'company_name':'TAU', 'company_description':'University'}, r=r, companies=companies)

TAU Added successfully!


In [31]:
# Operation 2 - Add a new job position
add_job({'job_title':'bi developer', 'location': 'Tel Aviv','requirements':['python','big data','mongodb'],'status':'open','publish_date':'01-02-2020'},'TAU', r=r, companies=companies)

job with id 1 was added to TAU jobs successfully!


In [32]:
# Operation 3 - Add a new application
new_application({'candidate_name':'laura', 'email':'laura@gmail.com','linkedin':'https://www.linkedin.com/in/laura/', 'skills': ['python','sql']},'01-02-2020 15:00:00', '1','TAU', r=r, companies=companies)

laura submited application for job number 1 at TAU


In [33]:
# Operation 4 - Update job status
update_job_status('TAU','1','open', r=r, companies=companies)

job number: 1 at TAU is now: open


In [34]:
# Operation 5 - Show latest 10 companies
show_latest_10_companies('lebron@gmail.com')

[]

In [48]:
# Operation 6 - Show number of jobs
show_number_of_jobs('Tel Aviv','bi developer')

0

# Tests 

In [ ]:
list(db['companies'].find({}))

In [ ]:
match = {'$match' : {'company_name':'TAU'}}
project = {'$project': { 'max_id': { '$size':'$jobs_list' }}}
# project = {'$project': { 'count': { '$size':'$jobs_list' }}}
res = companies.aggregate([match,project])
list(res)

In [ ]:
unwind = {"$unwind": "$jobs_list"}
project = {"$project":{
            'open_jobs':{
                "$cond": {"jobs_list.status":{"$eq":"open"}}
                },
            'closed_jobs':{
                "$cond": {"jobs_list.status":{"$eq":"close"}}
            }
    }
}
group = { "$group": {
            "_id": {
                "publish_date":"$jobs_list.publish_date",
                "company_name": "$company_name"
                },
            "open_jobs": {"$sum": 1}}}

res = companies.aggregate([unwind, project, group])
list(res)         

In [ ]:
res = companies.aggregate([
    { "$unwind": "$jobs_list"},
    { "$match": {"jobs_list.status": "open"} },
    { "$group": {"_id": "$jobs_list.publish_date", "open_jobs": {"$sum": 1}}}
    ])
list(res)

In [ ]:
new_application({'candidate_name':'laura', 'email':'laura@gmail.com','linkedin':'https://www.linkedin.com/in/laura/', 'skills': ['python','sql']},'14-02-2022 15:00:00', '3','TAU', r=r, companies=companies)

In [ ]:
r.zrange('candidate_applications:laura@gmail.com',0,-1,withscores=True)

In [ ]:
a = r.zrange('moti:luhim',0,-1,withscores=True)
b = [x for x in a if x[0]=='haim']
b

In [ ]:
r.keys('stud10:')

In [41]:
# r.get('ojOSet')

results = r.zrevrange(ojOSet, 0, -1, withscores=True)
for res in results:
    if res[0] == 'Tel Aviv:bi developer':
        print res[1]

1.0
